In [ ]:
 4.

# На основе алгоритма из предыдущего домашнего задания постройте стекинг (используйте 5-кратную кросс-валидацию) для всех моделей 
#с наилучшими подобранными параметрами. В качестве тренировочных данных используйте весь датасет train.csv, 
#а в качестве тестовых - весь датасет test.csv. Сделайте прогноз мета-алгоритма для test.csv.

In [142]:
import pandas as pd
import re
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Load in the train and test datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#print(train)
#print(test)

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

full_data = [train, test]

# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4


# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

test  = test.drop( ['PassengerId','Name', 'Ticket', 'Cabin', 'SibSp'], axis = 1)

#test_1=



targets = train['Survived']
data = train.drop('Survived', axis=1)






y_train=targets
x_train=data
x_test=test


y_test= targets.iloc[:418]



train=x_train
valid= x_test
train_true=y_train
valid_true=y_test






from sklearn.model_selection import (GridSearchCV,
                                     train_test_split,
                                     StratifiedKFold)
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score






lr_params = {'C': np.arange(0.5, 1, 0.1)}

skf = StratifiedKFold(n_splits=5, random_state=17,shuffle=True)


rfc_params = {'n_estimators': [70], # RandomForestClassifier
              'min_samples_leaf': [3]}



rf_model = RandomForestClassifier(random_state=17)
grid_search_rf = GridSearchCV(rf_model, rfc_params,cv=5)
rf_fit=grid_search_rf.fit(train, train_true)

brf_fitest_params = grid_search_rf.best_params_
best_score = grid_search_rf.best_score_
print('brf_fitest_params= ',brf_fitest_params,'best_score= ',best_score)


from sklearn.ensemble import GradientBoostingClassifier

gbc_params = {'learning_rate': [0.1]}


gbc_model = GradientBoostingClassifier(random_state=17)
grid_search_gbc= GridSearchCV(gbc_model, gbc_params,cv=5)
gbc_fit=grid_search_gbc.fit(train, train_true)

gbc_fitest_params = grid_search_gbc.best_params_
gbc_best_score = grid_search_gbc.best_score_
print('gbc_fitest_params= ',gbc_fitest_params,'gbc_best_score= ',gbc_best_score)


from sklearn import svm

svc_params = {'kernel': ['linear'], # SVC
             'C': [0.1]}

svc_model = svm.SVC(random_state=17)
grid_search_svc= GridSearchCV(svc_model, svc_params,cv=5)
svc_fit=grid_search_svc.fit(train, train_true)

svc_fitest_params = grid_search_svc.best_params_
svc_best_score = grid_search_svc.best_score_
print('svc_fitest_params= ',svc_fitest_params,'svc_best_score= ',svc_best_score)


from sklearn.linear_model import LogisticRegression


lr_params = {'C': [0.5]}

lr_model = LogisticRegression(random_state=17)
grid_search_lr= GridSearchCV(lr_model, lr_params,cv=5)
lr_fit=grid_search_lr.fit(train, train_true)

lr_fitest_params = grid_search_lr.best_params_
lr_best_score = grid_search_lr.best_score_
print('lr_fitest_params= ',lr_fitest_params,'lr_best_score= ',lr_best_score)





xgb_params = {'n_estimators':[95],
              'learning_rate': [0.4],
              'min_child_weight': [8],
              'subsample': [0.3],
              'reg_lambda':[1]  }


from xgboost import XGBClassifier



#meta_models = [rf_fit, lr_fit, gbc_fit, svc_fit]


# Определим базовые алгоритмы
meta_models = [rf_fit, lr_fit, gbc_fit, svc_fit]




brf_fitest_params=  {'min_samples_leaf': 3, 'n_estimators': 70} best_score=  0.8215366267026551
gbc_fitest_params=  {'learning_rate': 0.1} gbc_best_score=  0.8136965664427844
svc_fitest_params=  {'C': 0.1, 'kernel': 'linear'} svc_best_score=  0.8091770761408574
lr_fitest_params=  {'C': 0.5} lr_best_score=  0.8091833532107213


In [150]:


# # Создадим пустую матрицу для мета-признаков
meta_mtrx_m = np.empty((x_train.shape[0], len(meta_models)))

# # Получим предсказания от базовых алгоритмов на валидационном наборе
for n, model in enumerate(meta_models):
     meta_mtrx_m[:, n] = model.predict(x_train)



# # Создадим мета-алгоритм XGBoost
meta = XGBClassifier(random_state=17)

# # Создадим объект GridSearchCV
grid_search_meta = GridSearchCV(meta, xgb_params, cv=5)

# # Обучим мета-алгоритм на мета-признаках
grid_search_meta.fit(meta_mtrx_m, y_train)

# #print(valid_true )
#grid_search_meta.fit(meta_mtrx_m, valid_true)

# # Получим лучшие параметры
best_params = grid_search_meta.best_params_



meta_best_score = grid_search_meta.best_score_
print('meta__params= ',best_params,'meta_best_score= ',meta_best_score)






meta__params=  {'learning_rate': 0.4, 'min_child_weight': 8, 'n_estimators': 95, 'reg_lambda': 1, 'subsample': 0.3} meta_best_score=  0.8854811374050591


In [ ]:
# 5.

#С помощью нижеприведенной функции сформируйте файл посылки для соревнования и отправьте на Kaggle.

In [151]:
import pandas as pd



# Загрузите данные PassengerID
passenger_ids = pd.read_csv('test.csv')['PassengerId']
#print(passenger_ids)
# Сделайте прогнозы
predictions = meta_predict  # замените это фактическими прогнозами

#Создайте DataFrame с прогнозами и PassengerID
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': predictions
})
submission


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [154]:

def write_to_submission_file(predictions, passenger_ids, out_file='Submission.csv', columns=['PassengerID', 'Survived']):
    predicted_df = pd.DataFrame(np.array([passenger_ids,  meta_predict]).T, columns=columns)
    predicted_df.to_csv(out_file, index=False)
   #predicted_df.to_csv("C:/output.csv", index=False)
write_to_submission_file(predictions,passenger_ids) 

In [ ]:
# 6.

#Каков результат score, полученного на соревновании?

Ваш ответ: Score: 0.76315
